In [1]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malaysian-open-qa/resolve/main/hansard-qa.jsonl
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malaysian-open-qa/resolve/main/common-crawl-qa.jsonl

In [2]:
import instructor
import json
import random
from pydantic import BaseModel, Field
from enum import Enum
from typing import List

class QA(BaseModel):
    question: str
    answer: str
        
class QAS(BaseModel):
    qa: List[QA]

In [3]:
choices = json.dumps(instructor.openai_schema(QAS).openai_schema, indent=4)
print(choices)

{
    "name": "QAS",
    "description": "Correctly extracted `QAS` with all the required parameters with correct types",
    "parameters": {
        "$defs": {
            "QA": {
                "properties": {
                    "question": {
                        "type": "string"
                    },
                    "answer": {
                        "type": "string"
                    }
                },
                "required": [
                    "question",
                    "answer"
                ],
                "type": "object"
            }
        },
        "properties": {
            "qa": {
                "items": {
                    "$ref": "#/$defs/QA"
                },
                "type": "array"
            }
        },
        "required": [
            "qa"
        ],
        "type": "object"
    }
}


In [4]:
data = []

with open('/home/husein/ssd3/wikipedia-data/wikipedia-qa.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if not l['qa']:
            continue
        
        d = {
            'name': 'QAS',
            'arguments': {
                'qa': l['qa']['qa']
            }
        }
        data.append((l['paragraph'], d))
len(data)

44923

In [5]:
with open('hansard-qa.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        
        if not l['qa']:
            continue
        
        d = {
            'name': 'QAS',
            'arguments': {
                'qa': l['qa']['qa']
            }
        }
        data.append((l['paragraph']['original'], d))

In [6]:
with open('common-crawl-qa.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        
        if not l['qa']:
            continue
        
        d = {
            'name': 'QAS',
            'arguments': {
                'qa': l['qa']['qa']
            }
        }
        data.append((l['paragraph'], d))

In [7]:
len(data)

157120

In [8]:
templates = [
    'tukar ke JSON berdasarkan schema {schema}, teks `{text}`',
    'text `{text}`, convert to JSON using schema {schema}',
    'teks: {text}\n\ntukar ke JSON using schema {schema}',
    'convert to JSON using schema {schema}\n\ntext: {text}',
    '{text}\n\nJSON berdasarkan schema {schema}',
    'JSON berdasarkan schema {schema}\n\n{text}',
]

In [9]:
chat = []
for d in data:
    chat.append({
        'prompt_input': None,
        'input': random.choice(templates).format(schema = choices, text = d[0]),
        'output': json.dumps(d[1]),
    })

In [10]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [14]:
print(generate_and_tokenize_prompt(chat[2])['text'])

<s>[INST] teks: Galur () ialah bentuk tanah yang dibuat oleh air yang mengalir, menghakis tajam ke dalam tanah atau bahan lain yang relatifnya menghakis, biasanya di lereng bukit. Galur menyerupai parit besar atau lembah kecil, tetapi kedalaman dan lebarnya berukuran hingga puluhan meter. Ketika pembentukan saluran air sedang dalam proses, kadar aliran air boleh menjadi besar, menyebabkan tindakan pemotongan mendalam yang signifikan ke dalam substrat. Hakisan galur ialah proses yang membentuk galur. Lereng bukit adalah lebih rentan terhadap hakisan galur apabila mereka dibersihkan dari tumbuh-tumbuhan, melalui penebangan hutan, ragut lampau atau cara lain. Tanah yang terhakis mudah dibawa oleh air yang mengalir setelah terlepas daripada tanah, biasanya ketika hujan turun ketika ribut pendek dan kuat seperti ketika ribut petir. Galur boleh memanjang melalui hakisan menghulu pada takat terjun. Hakisan ini boleh terhasil daripada aliran antara serta larian permukaan. Galur mengurangkan pr

In [12]:
import json

with open('prepared-function-call-malaysian-open-qa.jsonl', 'w') as fopen:
    for l in chat:
        fopen.write(f'{json.dumps(l)}\n')